# SQL in Python Assignment

In [4]:
import pandas as pd
from sqlalchemy import *

### Connect to the personal database you created yesterday.
Remember the way the credentials work:
```
user = 'postgres' # Replace with your user, if different
pwd = 'pAsSwOrD' # Replace with your password
host = 'localhost'
port = '5432'
db = 'postgres' # Replace with your database name, if different
```

In [5]:
user = 'postgres'
pwd = '4Seasons**'
host = 'localhost'
port = '5432'
db = 'postgres'

engine = create_engine('postgresql://' + user + ':' + pwd + '@' + host + ':' + port + '/' + db)

### Obtain a list of tables that are in the database.

In [6]:
engine.table_names()

['vehicles', 'houseprices']

### Obtain a list of all the fields that are in the vehicles table along with the data type of each.

In [9]:
meta = MetaData()

vehicles = Table('vehicles', meta, autoload=True, autoload_with=engine)
fields = [(c.name, c.type) for c in vehicles.columns]
fields_df = pd.DataFrame(fields, columns=['Name', 'Type'])
fields_df

,Name,Type
0,id,BIGINT
1,make,TEXT
2,model,TEXT
3,year,BIGINT
4,class,TEXT
5,trans,TEXT
6,drive,TEXT
7,cyl,DOUBLE PRECISION
8,displ,DOUBLE PRECISION
9,fuel,TEXT


### Write a SELECT query to retrieve all records from the database and load them into a Pandas data frame.

In [10]:
sql = '''
SELECT * 
FROM vehicles;
'''

results = engine.execute(sql)
results_df = pd.DataFrame(results, columns=list(fields_df.Name))
results_df.head(10)

,id,make,model,year,class,trans,drive,cyl,displ,fuel,hwy,cty
0,27550,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4.0,2.5,Regular,17,18
1,28426,AM General,DJ Po Vehicle 2WD,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,4.0,2.5,Regular,17,18
2,27549,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6.0,4.2,Regular,13,13
3,28425,AM General,FJ8c Post Office,1984,Special Purpose Vehicle 2WD,Automatic 3-spd,2-Wheel Drive,6.0,4.2,Regular,13,13
4,1032,AM General,Post Office DJ5 2WD,1985,Special Purpose Vehicle 2WD,Automatic 3-spd,Rear-Wheel Drive,4.0,2.5,Regular,17,16
5,1033,AM General,Post Office DJ8 2WD,1985,Special Purpose Vehicle 2WD,Automatic 3-spd,Rear-Wheel Drive,6.0,4.2,Regular,13,13
6,3347,ASC Incorporated,GNX,1987,Midsize Cars,Automatic 4-spd,Rear-Wheel Drive,6.0,3.8,Premium,21,14
7,13309,Acura,2.2CL/3.0CL,1997,Subcompact Cars,Automatic 4-spd,Front-Wheel Drive,4.0,2.2,Regular,26,20
8,13310,Acura,2.2CL/3.0CL,1997,Subcompact Cars,Manual 5-spd,Front-Wheel Drive,4.0,2.2,Regular,28,22
9,13311,Acura,2.2CL/3.0CL,1997,Subcompact Cars,Automatic 4-spd,Front-Wheel Drive,6.0,3.0,Regular,26,18


### Create a table that contains all Audi vehicles with a city fuel efficiency greater than 18.

In [11]:
create = '''
CREATE TABLE IF NOT EXISTS audi AS
SELECT *
FROM vehicles
WHERE make = 'Audi'
AND cty > 18
'''

engine.execute(create)

### Delete all records from the temp table you just created.

In [12]:
delete = 'DELETE FROM audi;'
engine.execute(delete)

### Insert records from the vehicles table back into the table you created where the make is Audi and the city fuel efficiency is greater than 20.

In [13]:
insert = '''
INSERT INTO audi
SELECT *
FROM vehicles
WHERE make = 'Audi'
AND cty > 20
'''

engine.execute(insert)

### Update the drive field of the records in the table you created to change any "4-Wheel or All-Wheel Drive" values to "All-Wheel Drive."

In [14]:
update = '''
UPDATE audi
SET drive = 'All-Wheel Drive'
WHERE drive = '4-Wheel or All-Wheel Drive'
'''

engine.execute(update)

### Write a for loop that iterates over the list of makes provided below, selects the top 10 vehicles with the highest city fuel efficiency for each make, and then concatenates all the results together into a single Pandas data frame.

In [15]:
makes = ['Ford', 'Chevrolet', 'Toyota', 'Honda']

In [16]:
results = []

for make in makes:
    sql = "SELECT * FROM vehicles WHERE make = '" + make + "' ORDER BY cty DESC LIMIT 10"
    
    result = engine.execute(sql)
    results.append(pd.DataFrame(result, columns=list(fields_df.Name)))
    
results_df = pd.concat(results)